In [12]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from datetime import datetime
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [5]:
company_data = pd.read_csv("company_data.csv")
ticker_data = pd.read_csv("ticker_data.csv")

In [6]:
company_data.head()

,ticker,dimension,calendardate,datekey,reportperiod,lastupdated,accoci,assets,assetsavg,assetsc,...,sharesbas,shareswa,shareswadil,sps,tangibles,taxassets,taxexp,taxliabilities,tbvps,workingcapital
0,XOM,MRY,2018-12-31,2018-12-31,2018-12-31,2019-05-02,-19564000000,346196000000,349610000000,4.797300e+10,...,4233807170,4233807170,NaN,68.546,346196000000,0,9532000000,29856000000,81.769,-9.165000e+09
1,XOM,MRY,2017-12-31,2017-12-31,2017-12-31,2019-05-02,-16262000000,348691000000,346334750000,4.713400e+10,...,4237106077,4237106077,NaN,57.672,348691000000,0,-1174000000,29938000000,82.295,-1.063700e+10
2,XOM,MRY,2016-12-31,2016-12-31,2016-12-31,2019-05-02,-22239000000,330314000000,338740500000,4.141600e+10,...,4146693326,4146693326,NaN,50.188,330314000000,0,-406000000,36656000000,79.657,-6.222000e+09
3,XOM,MRY,2015-12-31,2015-12-31,2015-12-31,2019-05-02,-23511000000,336758000000,342160250000,4.262300e+10,...,4162938512,4162938512,NaN,59.873,336758000000,0,5415000000,39620000000,80.894,-1.135300e+10
4,XOM,MRY,2014-12-31,2014-12-31,2014-12-31,2019-05-02,-18957000000,349493000000,353469000000,5.291000e+10,...,4234528643,4234528643,NaN,97.281,349493000000,0,18015000000,44168000000,82.534,-1.172300e+10


In [7]:
# #transform company_data
# company_clean = company_data.reset_index(drop=True)
# company_cols = company_clean[["ticker","calendardate","assetturnover","currentratio","de","epsusd","grossmargin",
#                              "netmargin","pe","price"]]
# company_cols.head()

,ticker,calendardate,assetturnover,currentratio,de,epsusd,grossmargin,netmargin,pe,price
0,XOM,2018-12-31,0.830,0.840,0.770,4.88,0.349,0.072,13.853,68.19
1,XOM,2017-12-31,0.706,0.816,0.822,4.63,0.352,0.081,17.980,83.64
2,XOM,2016-12-31,0.614,0.869,0.935,1.88,0.360,0.038,47.740,90.26
3,XOM,2015-12-31,0.728,0.790,0.936,3.85,0.357,0.065,20.093,77.95
4,XOM,2014-12-31,1.165,0.819,0.966,7.60,0.302,0.079,12.038,92.45


In [8]:
#ticker data
ticker_data.head()

,table,permaticker,ticker,name,exchange,isdelisted,category,cusips,siccode,sicsector,...,currency,location,lastupdated,firstadded,firstpricedate,lastpricedate,firstquarter,lastquarter,secfilings,companysite
0,SF1,196290,A,Agilent Technologies Inc,NYSE,N,Domestic,00846U101,3826.0,Manufacturing,...,USD,California U.S.A,2019-05-30,2014-09-26,1999-11-18,2019-06-11,1997-06-30,2019-03-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.agilent.com
1,SF1,124392,AA,Alcoa Corp,NYSE,N,Domestic,013872106,3350.0,Manufacturing,...,USD,New York U.S.A,2019-05-09,2016-11-01,2016-11-01,2019-06-11,2014-12-31,2019-03-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.alcoa.com
2,SF1,122827,AAAB,Admiralty Bancorp Inc,NASDAQ,Y,Domestic,007231103,6022.0,Finance Insurance And Real Estate,...,USD,Florida U.S.A,2018-10-16,2017-09-09,1998-09-28,2003-01-29,1997-09-30,2002-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN
3,SF1,120538,AAAGY,Altana Aktiengesellschaft,NYSE,Y,ADR,02143N103,2834.0,Manufacturing,...,EUR,Jordan,2019-05-17,2018-02-13,2002-05-22,2010-08-12,2000-12-31,2005-12-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN
4,SF1,155760,AAAP,Advanced Accelerator Applications SA,NASDAQ,Y,ADR,00790T100,2834.0,Manufacturing,...,EUR,France,2017-07-17,2016-05-19,2015-11-11,2018-02-09,2013-12-31,2016-12-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN


In [81]:
# #transform ticker data
# ticker_cols = ticker_data[["ticker","name", "category","sicsector","currency","location","sector"]]
# ticker_cols.head()

,ticker,name,category,sicsector,currency,location,sector
0,A,Agilent Technologies Inc,Domestic,Manufacturing,USD,California U.S.A,Healthcare
1,AA,Alcoa Corp,Domestic,Manufacturing,USD,New York U.S.A,Basic Materials
2,AAAB,Admiralty Bancorp Inc,Domestic,Finance Insurance And Real Estate,USD,Florida U.S.A,Financial Services
3,AAAGY,Altana Aktiengesellschaft,ADR,Manufacturing,EUR,Jordan,Healthcare
4,AAAP,Advanced Accelerator Applications SA,ADR,Manufacturing,EUR,France,Healthcare


In [165]:
# #Create connection
# rds_connection_string = "root:##Kk26agro@127.0.0.1/company_db"
# engine = create_engine(f'mysql://{rds_connection_string}')

In [166]:
# engine.table_names()

['company']

In [167]:
# #export company_cols table into sql
# company_cols.to_sql(name='company', con=engine, if_exists='append', index=False)

In [168]:
# #export ticker_cols table into sql
# ticker_cols.to_sql(name='ticker', con=engine, if_exists='append', index=False)

In [9]:
#create engine from project2.sqlite
engine = create_engine("sqlite:///project2.sqlite")

In [10]:
engine

Engine(sqlite:///project2.sqlite)

In [13]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [14]:
# We can view all of the classes that automap found
Base.classes.keys()

[]

In [15]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

AttributeError: measurement